In [2]:
import sys
sys.path.append('..')

from models.MawiNet import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision
from torchvision.transforms import ToTensor
from torchvision import datasets
import matplotlib.pyplot as plt


import os
import numpy as np
from datetime import datetime as dt

model = MawiNetWithExitsCIFAR10()

print(model.backbone)
print(model.exits)

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (1): Sequential(
    (0): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
)
Sequential(
  (0): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=800, out_features=2, bias=True)
  )
  (1): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1024, out_features=2, bias=True)
  )
  (2): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1152, out_features=2, bias=True)
  )
)


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from pathlib import Path

files = Path('../../datasets/balanced/2019/VIEGAS/XX').iterdir()

df = pd.DataFrame()

for file in files:
    temp = pd.read_csv(file)
    df = pd.concat([df, temp])
    
df = df.drop(['MAWILAB_taxonomy', 'MAWILAB_distance', 'MAWILAB_nbDetectors', 'MAWILAB_label'], axis=1)

X = df.drop('class', axis=1)
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
data = []
for val in X_train.values:
    val = np.append(val, 0).reshape(7,7)
    data.append(val)
    
X_train = torch.FloatTensor(data).view(len(data), 1, 7, 7)
y_train = y_train.values

data = []
for val in X_test.values:
    val = np.append(val, 0).reshape(7,7)
    data.append(val)
    
X_test = torch.FloatTensor(data).view(len(data), 1, 7, 7)
y_test = y_test.values

train_data = []
for i in range(len(X_train)):
    train_data.append((X_train[i], y_train[i]))

test_data = []
for i in range(len(X_test)):
    train_data.append((X_test[i], y_test[i]))

/tmp/ipykernel_346463/3813123827.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  X_train = torch.FloatTensor(data).view(len(data), 1, 7, 7)


In [5]:
for i, (X_train, y_train) in enumerate(train_data):
    X_train = X_train
    y_train = y_train
    break 

x = X_train
print(x.shape)  
x = x.view(1,1,7,7)
print(x.shape)
x = model.backbone[0](x)
print(x.shape)
# x = model.exits[0](x)
# print(x.shape)
x = model.backbone[1](x)
print(x.shape)
# x = model.exits[1](x)
# print(x.shape)

x = model.backbone[2](x)
print(x.shape)
x = model.exits[2](x)
print(x.shape)





# x = nn.MaxPool2d(kernel_size=5, stride=2)(x)
# print(x.shape)

torch.Size([1, 7, 7])
torch.Size([1, 1, 7, 7])
torch.Size([1, 32, 5, 5])
torch.Size([1, 64, 4, 4])
torch.Size([1, 128, 3, 3])
torch.Size([1, 2])
